<hr/>

# Data Mining [EN.550.436]
**Tamás Budavári** - budavari@jhu.edu <br/>
**Class 11** - Oct 12, 2016

- Classification problems
- Naive Bayes Classifier
- Linear Discriminant Analysis
- Quadratic Discriminant Analysis

<hr/>

<h1><font color="darkblue">Classification</font></h1>

- Based on a **training set** of labeled points, assign class labels to unknown vectors in the **query set**.  

> **Training set**

>$T = \big\{ (\boldsymbol{x}_i, C_i) \big\}_{i=1}^N$ where $x_i\in \mathbb{R}^d$ and $C_i$ is the known class membership 

> **Query set**

>$Q = \big\{ \boldsymbol{x}_j \big\}_{j=1}^M$ where $x_j\in \mathbb{R}^d$ 

> For example,
> blood test results ($\boldsymbol{x}$) and sick/healty ($C$) - we want to predict if a new patient is sick based on the available measurements

- Similar to regression but with discrete categories to classify into...

### Classification Methods

- $k$-NN
- Naive Bayes
- Linear Discriminant Analysis
- Logistic regression
- Decisions trees
- Random forests
- Support Vector Machines

### The Iris Dataset

We'll use this data set available in [scikit-learn](http://scikit-learn.org/stable/index.html), see [this](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) page for details

### Other examples

More [exercises](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#examples-using-sklearn-neighbors-kneighborsclassifier) such as classification of [digits](http://scikit-learn.org/stable/auto_examples/exercises/digits_classification_exercise.html) are available at http://scikit-learn.org

### Naive Bayes Classifier

- Using Bayes' rule to infer discrete classes $C_k$ for a given $\boldsymbol{x}$ set of features

>$\displaystyle P(C_k|\boldsymbol{x}) = \frac{1}{Z}\ \pi(C_k)\,{\cal{}L}_{\boldsymbol{x}}(C_k)$ 


- Naively assuming the features are independent 

>$\displaystyle {\cal{}L}_{\boldsymbol{x}}(C_k) = \prod_{\alpha}^d p(x_{\alpha}|C_k)$ 


### Naive Bayes: Learning

- Say for Gaussian likelihoods, we simply estimate the sample mean and variance of all features for each class $k$

>$\displaystyle p(x_{\alpha}|C_k) = G(x_{\alpha};\mu_k, \sigma^2_k)$

- We have to also pick some prior for the classes

> Using uniform or based on frequency of points in training set



### Naive Bayes: Estimation

- Look for maximum of the posterior


>$\displaystyle \hat{k} =  \mathrm{arg}\max_k \left[ \pi_k \prod_{\alpha}^d G(x_{\alpha};\mu_k, \sigma^2_k)\right]$ 


In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [15]:
from sklearn import datasets
iris = datasets.load_iris()   #load iris

#实际操作中，先读取数据为矩阵

In [11]:
# unique known classes in training set
classes = np.unique(iris.target)   #获取不同类别
print 'There are %d classes:' % len(classes), classes

# calc feature means and variances for each class
param = dict() # we save them in this dictionary，数据类别
for k in classes:
    members = (iris.target == k) # boolean array，判断每个样本是否属于k
    num = members.sum()    # True:1, False:0，类别成员总计
    prior = num / float(iris.target.size)   #prior
    X = iris.data[members,:] # slice out members，取出第k类数据
    mu = X.mean(axis=0)      # calc mean
    X -= mu   #center k type
    var = (X*X).sum(axis=0) / (X[:,0].size-1)   #X*X对应元素相乘，得到与X一致形状的矩阵,修改为第0列
    param[k] = (num, prior, mu, var) # save results
    if True: print k, mu, var   #print results

There are 3 classes: [0 1 2]
0 [ 5.006  3.418  1.464  0.244] [ 0.12424898  0.14517959  0.03010612  0.01149388]
1 [ 5.936  2.77   4.26   1.326] [ 0.26643265  0.09846939  0.22081633  0.03910612]
2 [ 6.588  2.974  5.552  2.026] [ 0.40434286  0.10400408  0.30458776  0.07543265]


In [13]:
# init predicted values
k_pred = -1 * ones(iris.target.size)

# evaluate posterior for each point and find maximum
for i in range(iris.target.size):   #遍历每个个体
    pmax, kmax = -1, None   # initialize to nonsense values
    for k in classes:
        num, prior, mu, var = param[k]
        diff = iris.data[i,:] - mu
        d2 = diff*diff / var / 2    #d2=diff^2/(2*var)
        p = prior * np.exp(-d2.sum())   #posterior Guassian,都加起来是认为4个特征是4个独立变量,故直接乘起来等于概率
        if p > pmax:   #diff越大,p越小,取得的实际上是最接近的类别
            pmax = p
            kmax = k
    k_pred[i] = kmax

print("Number of mislabeled points out of a total %d points : %d"
      % (iris.target.size, (iris.target!=k_pred).sum()))   #用已知的数据检验某种算法

Number of mislabeled points out of a total 150 points : 8


array([ 0.58532504,  0.00324987,  0.33537789,  0.33855365])

In [5]:
# run sklearn's version - read up on differences if interested
from sklearn.naive_bayes import GaussianNB   #另一个工具箱,naive bayes，GUASSIAN

gnb = GaussianNB()
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)

print("Number of mislabeled points out of a total %d points : %d"   #算法肯定有修正
      % (iris.target.size, (iris.target!=y_pred).sum()))

Number of mislabeled points out of a total 150 points : 6


### Assumptions

- Features are automatically treated correctly relative to each other

> For example, measuring similar things in different units? 1m vs 1mm

> The estimated mean and variance puts them on a meaningful scale

- Independence is a strong assumption and for no good reason

> Actually it helps with the computational costs...

### Exercise: Naive Bayes

- Use the provided [training](files/Class12-Train.csv) and [query](files/Class12-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)

> **Query** set only has 2 columns of ($x_i$, $y_i$)

- Place post-it on your laptop to indicate your status and if you need help


### Full Covariance Matrix

- Estimate the full covariance matrix for the classes

>$\displaystyle {\cal{}L}_{\boldsymbol{x}}(C_k) =  G(\boldsymbol{x};\mu_k, \Sigma_k)$

> Handles correlated features well

- Consider binary problem with 2 classes

> Taking the negative logarithm of the likelihoods we compare

>$\displaystyle (\boldsymbol{x}\!-\!\boldsymbol{\mu}_1)^T\,\Sigma_1^{-1}(\boldsymbol{x}\!-\!\boldsymbol{\mu}_1) + \ln|\Sigma_1|$ vs.

>$\displaystyle (\boldsymbol{x}\!-\!\boldsymbol{\mu}_2)^T\,\Sigma_2^{-1}(\boldsymbol{x}\!-\!\boldsymbol{\mu}_2) + \ln|\Sigma_2|$

> If the difference is lower than a threshold, we classify it accordingly

- This is called **Quadratic Discriminant Analysis**

### Same Covariance Matrix

- When $\Sigma_1=\Sigma_2=\Sigma$, the quadratic terms cancel from the difference
 
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma^{-1}(x\!-\!\mu_1) $ 
>$\displaystyle -\ (x\!-\!\mu_2)^T\,\Sigma^{-1}(x\!-\!\mu_2) $

- Hence this is called **Linear Discriminant Analysis**

> Fewer parameters to estimate during the learning process

> Good, if we don't have enough data, for example...

> Think linear vs quadratic fitting and how you decide between those

### Exercise: QDA and LDA

- Use the provided [training](files/Class12-Train.csv) and [query](files/Class12-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)

> **Query** set only has 2 columns of ($x_i$, $y_i$)

- Place post-it on your laptop to indicate your status and if you need help


### Done already?

- Visualize the results in the 2D features space
- Make these simple codes run faster by getting rid of the 'for' loops, etc.
